# Data Extraction   
---
# 1. Introduction

Here, I demonstrate the **Extract (E)** phase of the ETL process.  
The purpose of this step is to **load, inspect, and validate** the *Airline Tweets dataset*, which contains customer feedback directed at major airlines on Twitter(X).

This extraction process ensures that the data is **properly understood**, **well structured**, and **ready for transformation and analysis** (as detailed later in *etl_transform.ipynb*).  

### Tasks Performed
1. Loading both the **raw dataset** (`raw_data.csv`) and the **incremental dataset** (`incremental_data.csv`) from the `data/` directory.  
2. Performing exploratory data analysis to understand the datasets’ structure and characteristics.  
3. Identifying and documenting key data quality issues such as missing values, duplicates, and inconsistent types.  
4. Validating and ensuring column consistency between both datasets.  
5. Confirm readiness for transformation by verifying schema alignment and documenting observations.

---

In [45]:
# 1. Import Libraries

import pandas as pd
import numpy as np
import os

# Configure display options
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

---

## 2. Loading the Raw and Prepared Incremental Dataset  

I now load both the **raw** and **incremental** datasets, perform exploratory checks, and identify data quality issues such as missing values, duplicates, and inconsistent data types.  
The objective is to ensure both datasets are clean, structurally consistent, and ready for integration during the transformation stage. 

---

In [46]:
# Verify the data directory exists
base_dir = os.getcwd()
data_dir = os.path.join(base_dir, "data")

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print("Data directory verified at:", data_dir)

# Preview the first few rows
df.head()

Data directory verified at: /Users/nathanomenge/Desktop/ET_Exam_Nathan_637/data


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have littl...",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


### Load Datasets

I now load both the **raw dataset (`raw_data.csv`)** and the **incremental dataset (`incremental_data.csv`)** into Pandas DataFrames.

The raw dataset contains the original airline tweets, while the incremental dataset subsets the raw dataset to simulate newly acquired tweets.

In [47]:
# Define dataset paths
raw_data_path = os.path.join(data_dir, "raw_data.csv")
incremental_data_path = os.path.join(data_dir, "incremental_data.csv")

# Subsetting Raw dataset to Create Incremental Dataset
# Convert tweet_created to datetime for sorting
df_raw["tweet_created"] = pd.to_datetime(df_raw["tweet_created"], errors="coerce")

# Subset the most recent 2000 records to simulate new incremental data
df_incremental = (
    df_raw.sort_values("tweet_created", ascending=False)
          .head(2000)
          .copy()
)

# Save the incremental dataset
incremental_data_path = os.path.join(data_dir, "incremental_data.csv")
df_incremental.to_csv(incremental_data_path, index=False)

print(f"Incremental dataset created and saved at: {incremental_data_path}")
print(f"Incremental dataset shape: {df_incremental.shape}")


# Visual confirmation of dataset similarity
print("\n--- RAW DATA PREVIEW ---")
display(df_raw.head(3))

print("\n--- INCREMENTAL DATA PREVIEW ---")
display(df_incremental.head(3))

Incremental dataset created and saved at: /Users/nathanomenge/Desktop/ET_Exam_Nathan_637/data/incremental_data.csv
Incremental dataset shape: (2000, 15)

--- RAW DATA PREVIEW ---


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52-08:00,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59-08:00,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48-08:00,Lets Play,Central Time (US & Canada)



--- INCREMENTAL DATA PREVIEW ---


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
8966,570310600460525568,negative,0.6292,Flight Booking Problems,0.3146,US Airways,NaN,jhazelnut,NaN,0,@USAirways is there a better time to call? My flight is on Friday and I need to change it. Worried I may be on hold...,NaN,2015-02-24 11:53:37-08:00,NaN,NaN
8967,570310144459972608,negative,1.0000,Customer Service Issue,1.0000,US Airways,NaN,GAKotsch,NaN,0,@USAirways and when will one of these agents be available to speak?,NaN,2015-02-24 11:51:48-08:00,NaN,Atlantic Time (Canada)
6746,570309345281486848,positive,0.6469,NaN,NaN,Delta,NaN,jaxbra,NaN,0,@JetBlue Yesterday on my way from EWR to FLL just after take-off. :)\n#wheelsup #JetBlueSoFly http://t.co/9xkiy0Kq2j,NaN,2015-02-24 11:48:38-08:00,"east brunswick, nj",Atlantic Time (Canada)


In [48]:
# --- STRUCTURAL OVERVIEW OF BOTH DATASETS ---

print("\n--- RAW DATASET STRUCTURE ---")
df_raw.info()

print("\n--- INCREMENTAL DATASET STRUCTURE ---")
df_incremental.info()

# Summary statistics for numerical columns
print("\n--- RAW DATASET SUMMARY STATISTICS ---")
display(df_raw.describe())

print("\n--- INCREMENTAL DATASET SUMMARY STATISTICS ---")
display(df_incremental.describe())


--- RAW DATASET STRUCTURE ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype                    
---  ------                        --------------  -----                    
 0   tweet_id                      14640 non-null  int64                    
 1   airline_sentiment             14640 non-null  object                   
 2   airline_sentiment_confidence  14640 non-null  float64                  
 3   negativereason                9178 non-null   object                   
 4   negativereason_confidence     10522 non-null  float64                  
 5   airline                       14640 non-null  object                   
 6   airline_sentiment_gold        40 non-null     object                   
 7   name                          14640 non-null  object                   
 8   negativereason_gold           32 non-null     object                   
 9   retweet_

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000



--- INCREMENTAL DATASET SUMMARY STATISTICS ---


,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,2.000000e+03,2000.000000,1411.000000,2000.000000
mean,5.701924e+17,0.891243,0.618679,0.060000
std,9.219932e+13,0.166962,0.338251,0.355617
min,5.700283e+17,0.336800,0.000000,0.000000
25%,5.700922e+17,0.685475,0.354450,0.000000
50%,5.702232e+17,1.000000,0.666700,0.000000
75%,5.702741e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,5.000000


### 3. Structural Overview Discussion

Both the **raw dataset** (`raw_data.csv`) and the **incremental dataset** (`incremental_data.csv`) contain **15 columns** with matching names, order, and data types.  

The columns represent tweet metadata, sentiment classification, and contextual information:
- **Identifiers and metadata:** `tweet_id`, `tweet_created`, `retweet_count`
- **Sentiment information:** `airline_sentiment`, `airline_sentiment_confidence`, `negativereason`, `negativereason_confidence`
- **Categorical and descriptive characteristics:** `airline`, `name`, `user_timezone`, and `tweet_location`

**Key Observations**
1. Both datasets maintain consistent data types — categorical fields stored as `object`, numerical metrics as `float64` or `int64`, and the timestamp column (`tweet_created`) correctly parsed as `datetime64[ns]`.  
2. Summary statistics show similar ranges and distributions for numeric fields such as `airline_sentiment_confidence`, `negativereason_confidence`, and `retweet_count`, confirming statistical consistency between the two datasets.  
3. The incremental dataset (2,000 records) is a chronologically recent subset of the raw data, preserving the same schema and integrity needed for downstream ETL transformation.  

This confirms that both datasets are now **ready for quality inspection**

---
## 3. Data Quality Checks

In [49]:
# 1. Missing Values
print("=== MISSING VALUES (RAW DATASET) ===\n")
missing_raw = df_raw.isnull().sum()
print(missing_raw[missing_raw > 0])

print("\n=== MISSING VALUES (INCREMENTAL DATASET) ===\n")
missing_incremental = df_incremental.isnull().sum()
print(missing_incremental[missing_incremental > 0])

# 2. Duplicate Records
print("\n=== DUPLICATE RECORDS ===")
duplicates_raw = df_raw.duplicated().sum()
duplicates_incremental = df_incremental.duplicated().sum()

print(f"Raw dataset duplicate records: {duplicates_raw}")
print(f"Incremental dataset duplicate records: {duplicates_incremental}")

# 3. Data Type Validation (Quick Cross-Check)
print("\n=== DATA TYPE CONSISTENCY ===")
print("RAW dataset types:\n", df_raw.dtypes)
print("\nINCREMENTAL dataset types:\n", df_incremental.dtypes)

=== MISSING VALUES (RAW DATASET) ===

negativereason                5462
negativereason_confidence     4118
airline_sentiment_gold       14600
negativereason_gold          14608
tweet_coord                  13621
tweet_location                4733
user_timezone                 4820
dtype: int64

=== MISSING VALUES (INCREMENTAL DATASET) ===

negativereason                791
negativereason_confidence     589
airline_sentiment_gold       1995
negativereason_gold          1997
tweet_coord                  1877
tweet_location                726
user_timezone                 693
dtype: int64

=== DUPLICATE RECORDS ===
Raw dataset duplicate records: 36
Incremental dataset duplicate records: 36

=== DATA TYPE CONSISTENCY ===
RAW dataset types:
 tweet_id                                            int64
airline_sentiment                                  object
airline_sentiment_confidence                      float64
negativereason                                     object
negativereason_confi

### Data Quality Checks Discussion

#### (a) Missing Values
Both datasets show missing values in several columns, reflecting the informal and user-generated nature of Twitter data:

- **`negativereason`** and **`negativereason_confidence`** have null entries since not every tweet provides a complaint or reason for dissatisfaction.  
- **`tweet_coord`**, **`tweet_location`**, and **`user_timezone`** are also partially missing .
- **`airline_sentiment_gold`** and **`negativereason_gold`** are almost fully empty.

#### (b) Duplicate Records
Both datasets contain **36 duplicate rows**. These duplicates are likely due to:

These duplicates will be removed during transformation.

#### (c) Data Type Consistency
All columns exhibit consistent data types across datasets:
- **Categorical / textual:** stored as `object`
- **Numerical:** stored as `float64` or `int64`
- **Temporal:** `tweet_created` correctly parsed as `datetime64[ns]`

I found no structural or type discrepancies, proving both datasets are now ready for the next validation step.

---
## 4. Merging and Validating the Combined Dataset

The next thing I did was to verify that both the **raw** and **incremental** datasets share a consistent structure and can be merged seamlessly if needed during transformation.

This step validates:
1. Column alignment — ensuring both datasets have identical structures.
2. Row count expectations — confirming the merge produces the correct total records.
3. Overlap and duplicates — identifying any repeated records across datasets.



In [50]:
# Compare column structures between the two datasets
raw_columns = set(df_raw.columns)
incremental_columns = set(df_incremental.columns)

print("Columns only in raw dataset:", raw_columns - incremental_columns)
print("Columns only in incremental dataset:", incremental_columns - raw_columns)

if raw_columns == incremental_columns:
    print("\nColumn structure is consistent across both datasets.\n")
else:
    print("\nColumn mismatch detected — review column alignment before transformation.\n")

#  Validate the merge
df_combined = pd.concat([df_raw, df_incremental], ignore_index=True)

# Compute and display validation metrics
expected_total = len(df_raw) + len(df_incremental)
duplicates_after_merge = df_combined.duplicated().sum()

print(f"Combined dataset shape: {df_combined.shape}")
print(f"Expected total records (before removing duplicates): {expected_total}")
print(f"Total duplicate records after merge: {duplicates_after_merge}")

# Confirm readiness for transformation
print("\n Merge validated — schema and structure are consistent.")

Columns only in raw dataset: set()
Columns only in incremental dataset: set()

Column structure is consistent across both datasets.

Combined dataset shape: (16640, 15)
Expected total records (before removing duplicates): 16640
Total duplicate records after merge: 2036

 Merge validated — schema and structure are consistent.


### Merge Validation Discussion

The merge validation confirms that:
- Both datasets share identical column structures and compatible data types.
- The combined dataset shape aligns with expectations.
- A higher duplicate count (2,036) arises due to overlap between the incremental subset and raw dataset, as expected since the incremental sample was derived from the latest 2,000 records.

This confirms that both datasets are **schema-aligned**, **structurally consistent**, and **ready for transformation** in the next phase (`etl_transform.ipynb`).

---
# 5. Saving Validated Data for Transformation

Having confirmed the structural integrity and schema alignment of both datasets, I now save the validated copies to the `data/` directory.  
These files will serve as clean, ready-to-transform inputs for the next notebook (`etl_transform.ipynb`).





In [51]:
# Save validated datasets to the data folder
raw_save_path = os.path.join(data_dir, "raw_data.csv")
incremental_save_path = os.path.join(data_dir, "incremental_data.csv")

df_raw.to_csv(raw_save_path, index=False)
df_incremental.to_csv(incremental_save_path, index=False)

print(f"Validated raw dataset saved at: {raw_save_path}")
print(f"Validated incremental dataset saved at: {incremental_save_path}")

Validated raw dataset saved at: /Users/nathanomenge/Desktop/ET_Exam_Nathan_637/data/raw_data.csv
Validated incremental dataset saved at: /Users/nathanomenge/Desktop/ET_Exam_Nathan_637/data/incremental_data.csv
